This is a notebook to parse the neighborhood and postal codes for Toronto as part of the Coursera Capstone Project Week 3 assignment.

We add the latitude and longitude to the previous dataframe containing the postal, borough, neighborhoods of Toronto.

First we load the needed libraries.

In [3]:
# Import numpy and pandas
import numpy as np
import pandas as pd

# install beautifulsoup and lxml
!pip install beautifulsoup4
!pip install lxml

# import beautifulsoup and requests
from bs4 import BeautifulSoup
import requests

print('All imported correctly')

All imported correctly


Then we perform the same steps that we did on P1 - Dataframe - Clean notebook to scrape the neighborhood data from Wikipedia.

In [46]:
# this loads the values to the dataframe (refer to P1 - Dataframe - Clean notebook for more details)
# set URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

# gets the source code of the website
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

# prints the source code in a more readable format
# print(soup.prettify())

# create new dataframe with column names
column_names = ['Postcode','Borough','Neighborhood']
table = pd.DataFrame(columns=column_names)

# loops through the webpage source code to find the table entries with 'tr' and 'td' markers and adds them to the dataframe
for entry in soup.find_all('tr', class_=None):
        
    try:
        postcode    =entry.find_all('td', class_=None)[0].text
        borough     =entry.find_all('td', class_=None)[1].text
        neighborhood=entry.find_all('td', class_=None)[2].text
        table = table.append({'Postcode':postcode,'Borough':borough,'Neighborhood':neighborhood},ignore_index=True)

    except:
        continue

# copy table to table2 (for cleaning)
table2=table.copy()

# remove last 3 rows with unrelated data
table2=table2[:-3]

# remove \n from neighborhood values
table2.replace(r'\n','',regex=True,inplace=True)

# copy table2 to table3 for removal of unassigned rows
table3=table2.copy()

# drop unassigned boroughs
table3.drop(table3.index[table3['Borough']=='Not assigned'],inplace=True)

# reset the count of the index so as not to skip the dropped rows
table3.reset_index(drop=True,inplace=True)

# set unassigned neighborhoods to be the same value as that of the borough
for index in table3[table3['Neighborhood']=='Not assigned'].index:
#     print("Value to be replaced: ", table3.loc[index,'Neighborhood'])
#     print("Value to replace with: ", table3.loc[index,'Borough'])
    table3.loc[index,'Neighborhood']=table3.loc[index,'Borough']

# merges neighborhood values for the same postal code and borough
table3=table3.groupby(['Postcode','Borough'])['Neighborhood'].apply(lambda tags: ', '.join(tags)).to_frame().reset_index()

# prints the final shape of the dataframe
print("Shape is: ", table3.shape)

# display final table
table3.head(10)

Shape is:  (103, 3)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


This is our attempt at using geocoder to get the latitude and longitude. However, after several iterations with None returned, we used the csv instead.

In [47]:
# install geocoder
# !pip install geocoder

# import geocoder
# import geocoder

# initialize your variable to None
# lat_lng_coords = None

# add latitude and longitude columns to dataframe
# table3 = table3.reindex(table3.columns.tolist() + ['Latitude','Longitude'], axis=1)
# i=0
# j=0

# loop until you get the coordinates for postal codes in dataframe
# for index in table3['Postcode'].index:
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(table3.loc[index,'Postcode']))
#         lat_lng_coords = g.latlng
#         j=j+1
#         print("Run: ",j)
#     table3.loc[index,'Latitude'] = lat_lng_coords[0]
#     table3.loc[index,'Longitude'] = lat_lng_coords[1]
#     i=i+1
# print("Row: ", i, " for postcode: ", table3.loc[index,'Postcode'])
# print("Latitude: ", table3.loc[index,'Latitude'])
# print("Longitude: ", table3.loc[index,'Longitude'])

# table3.head(15)

These next steps load the latitude and longitude data from the csv file provided and use the merge function to combine the 2 dataframes.

In [49]:
# load the latitude and longitude data from csv file
latlng_file = 'https://cocl.us/Geospatial_data'
latlng = pd.read_csv(latlng_file)
latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [50]:
# remove latitude and longitude columns from dataframe
table3=table3.iloc[:,0:3]

# use merge to get latitude and longitude data on main dataframe
table3 = pd.merge(table3, latlng, how='left',
        left_on='Postcode', right_on='Postal Code')

# remove the duplicate postal code column from csv file
table3.drop('Postal Code',axis=1,inplace=True)

table3.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [52]:
# confirm all columns have data for the same number of rows
table3.describe(include='all')

,Postcode,Borough,Neighborhood,Latitude,Longitude
count,103,103,103,103.000000,103.000000
unique,103,11,102,NaN,NaN
top,M1P,North York,Queen's Park,NaN,NaN
freq,1,24,2,NaN,NaN
mean,NaN,NaN,NaN,43.704608,-79.397153
std,NaN,NaN,NaN,0.052463,0.097146
min,NaN,NaN,NaN,43.602414,-79.615819
25%,NaN,NaN,NaN,43.660567,-79.464763
50%,NaN,NaN,NaN,43.696948,-79.388790
75%,NaN,NaN,NaN,43.745320,-79.340923
